# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saudarkrokur,65.7461,-19.6394,12.70,70,90,3.70,IS,1722810134
1,1,gongchangling,41.1167,123.4500,23.98,98,98,2.28,CN,1722810134
2,2,taroa,8.7058,171.2275,28.44,78,92,6.19,MH,1722810134
3,3,constantia,44.1833,28.6500,19.38,65,14,6.23,RO,1722810135
4,4,anadyr,64.7500,177.4833,5.28,100,100,8.00,RU,1722810135


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)

# Display the map
plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=30)\
                            &(city_data_df['Max Temp']>=15)\
                            &(city_data_df['Cloudiness']>= 10)\
                            &(city_data_df['Country']!='US')\
                            &(city_data_df['Country']!='AU')\
                            &(city_data_df['Country']!='NZ')\
                            &(city_data_df['Humidity']<= 40),:]
print(len(ideal_df))

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head()

15


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,shubarshi,48.5872,57.1869,22.79,37,80,1.54,KZ,1722810135
61,61,zhanaozen,43.3000,52.8000,28.36,34,59,6.93,KZ,1722810143
62,62,la ronge,55.1001,-105.2842,21.14,30,100,1.54,CA,1722810144
80,80,cosquin,-31.2451,-64.4656,15.69,40,22,2.24,AR,1722810147
111,111,korla,41.7597,86.1469,28.62,30,37,1.73,CN,1722810151


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
7,shubarshi,KZ,48.5872,57.1869,37,
61,zhanaozen,KZ,43.3000,52.8000,34,
62,la ronge,CA,55.1001,-105.2842,30,
80,cosquin,AR,-31.2451,-64.4656,40,
111,korla,CN,41.7597,86.1469,30,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
shubarshi - nearest hotel: No hotel found
zhanaozen - nearest hotel: No hotel found
la ronge - nearest hotel: La Ronge Hotel and Suites
cosquin - nearest hotel: Hostería La Serrana
korla - nearest hotel: Silver Star Hotel
thompson - nearest hotel: Thompson Inn
laojunmiao - nearest hotel: No hotel found
buka - nearest hotel: No hotel found
mutuali - nearest hotel: No hotel found
whitehorse - nearest hotel: Town & Mountain Hotel
justo daract - nearest hotel: No hotel found
kampene - nearest hotel: No hotel found
aral - nearest hotel: No hotel found
beyneu - nearest hotel: Johan yu
ribas do rio pardo - nearest hotel: Hotel São Francisco


,City,Country,Lat,Lng,Humidity,Hotel Name
7,shubarshi,KZ,48.5872,57.1869,37,No hotel found
61,zhanaozen,KZ,43.3000,52.8000,34,No hotel found
62,la ronge,CA,55.1001,-105.2842,30,La Ronge Hotel and Suites
80,cosquin,AR,-31.2451,-64.4656,40,Hostería La Serrana
111,korla,CN,41.7597,86.1469,30,Silver Star Hotel
118,thompson,CA,55.7435,-97.8558,28,Thompson Inn
125,laojunmiao,CN,39.8333,97.7333,22,No hotel found
151,buka,UZ,40.8108,69.1986,25,No hotel found
184,mutuali,MZ,-14.8706,37.0044,37,No hotel found
230,whitehorse,CA,60.7161,-135.0538,30,Town & Mountain Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
plot_4 = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
plot_4

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)